## Running a Workflow on a Seven Bridges WES server
I'm setting out to use the SevenBridges WES client to run samtools stats on a cram file. The instructions described here https://docs.cancergenomicscloud.org/docs/run-a-workflow are the starting point for how to do this.


In [12]:
#from fasp.workflow import sbWESClient
from fasp.workflow import sbcgcWESClient

cl = sbcgcWESClient('forei/CNest', debug=True)

In [13]:
cl.getRuns()

{
   "runs": [
      {
         "state": "COMPLETE",
         "run_id": "5c5e52df-2950-436f-a280-2b3358fc8260"
      },
      {
         "state": "COMPLETE",
         "run_id": "bf1dfafa-a929-4c47-a774-da12d345b3e0"
      },
      {
         "state": "COMPLETE",
         "run_id": "0f19dc9d-d352-4d51-a201-1872b112429e"
      },
      {
         "state": "COMPLETE",
         "run_id": "edce195f-c5d0-47d3-a4c7-a69404ecbabd"
      },
      {
         "state": "COMPLETE",
         "run_id": "7296d085-761f-422f-957c-3f05647827e3"
      },
      {
         "state": "UNKNOWN",
         "run_id": "56184541-cbc1-4977-af01-0acaa52a5ac9"
      },
      {
         "state": "EXECUTOR_ERROR",
         "run_id": "825a7ac8-58c5-44c8-9eaf-706099eef979"
      },
      {
         "state": "UNKNOWN",
         "run_id": "a0388a6b-db1a-4f1c-bacf-b66de11cc308"
      },
      {
         "state": "CANCELED",
         "run_id": "a5cb0f32-d497-423a-b5c3-3f21f6e86b4b"
      },
      {
         "state": "COMPLETE"

{
   "runs": [
      {
         "state": "UNKNOWN",
         "run_id": "493cf8d1-9dfc-4f60-99d6-d625e7993397"
      },
      {
         "state": "UNKNOWN",
         "run_id": "e32df71d-17ba-4e1b-ad0f-dd3f0a5dd1af"
      },
      {
         "state": "EXECUTOR_ERROR",
         "run_id": "65bc7d90-7924-4bac-963c-c3cbf981ef4e"
      },
      {
         "state": "UNKNOWN",
         "run_id": "6052f084-03bc-49e3-a60d-69beb46747a6"
      },
      {
         "state": "UNKNOWN",
         "run_id": "d568c91b-84b0-4c74-b1cc-bd95c29ad635"
      },
      {
         "state": "UNKNOWN",
         "run_id": "a47bec7e-eab1-40f3-ba56-8f3b008209a6"
      },
      {
         "state": "UNKNOWN",
         "run_id": "675b97ef-153f-445a-ad0a-559ab4c66406"
      },
      {
         "state": "EXECUTOR_ERROR",
         "run_id": "9d07fa23-1a84-4fef-9415-bf0385ee4d5c"
      },
      {
         "state": "UNKNOWN",
         "run_id": "26c1056c-fb92-478d-83e5-40d7174adcf6"
      },
      {
         "state": "UNKNOWN"

{
   "runs": [
      {
         "state": "COMPLETE",
         "run_id": "e0fd8109-93e4-4efa-b774-39a15c6ea872"
      },
      {
         "state": "COMPLETE",
         "run_id": "9025d17a-ad1b-45af-b67a-f38a6dd90259"
      },
      {
         "state": "COMPLETE",
         "run_id": "14244346-b60b-41d4-99fb-56295ed6d773"
      },
      {
         "state": "COMPLETE",
         "run_id": "f50dab9a-c40e-4fc6-8a97-415704d46351"
      },
      {
         "state": "COMPLETE",
         "run_id": "4c99da85-7b26-4326-8bce-868896ddfc9d"
      },
      {
         "state": "COMPLETE",
         "run_id": "93ed48b3-d5a7-4910-8764-5d0419228002"
      },
      {
         "state": "COMPLETE",
         "run_id": "e25f2337-65b4-433b-ad2c-cd352da80602"
      },
      {
         "state": "COMPLETE",
         "run_id": "ec318962-e576-4673-8b20-ff9cdfb0502e"
      },
      {
         "state": "COMPLETE",
         "run_id": "f05d7748-6be3-4f9b-aef2-704684af0bf6"
      },
      {
         "state": "COMPLETE",
  

{
   "runs": [
      {
         "state": "UNKNOWN",
         "run_id": "ab75e751-1b50-4b19-8a79-1d2094a448ab"
      },
      {
         "state": "UNKNOWN",
         "run_id": "558aae94-636b-4295-9175-22208375247a"
      },
      {
         "state": "COMPLETE",
         "run_id": "fe7e15d0-ba40-490e-94d3-81a0f00198b3"
      },
      {
         "state": "COMPLETE",
         "run_id": "d25c3eb6-933d-4362-b9aa-4ccfa84ce168"
      },
      {
         "state": "COMPLETE",
         "run_id": "5c799462-09ab-47f6-82c9-034fde7674ac"
      },
      {
         "state": "COMPLETE",
         "run_id": "ad52bd09-b70e-4ff8-af76-bc860996304f"
      },
      {
         "state": "COMPLETE",
         "run_id": "11b54fff-2d33-401c-b5c8-35cfde4919e4"
      },
      {
         "state": "COMPLETE",
         "run_id": "5731a48f-d908-45e2-9f9c-19b1952bf866"
      },
      {
         "state": "COMPLETE",
         "run_id": "3ce46fba-bbc4-415c-8833-ea6075dbe77a"
      },
      {
         "state": "COMPLETE",
    

{
   "runs": [
      {
         "state": "COMPLETE",
         "run_id": "9d7ef799-c638-4505-9c4b-d5139e4c4fbc"
      },
      {
         "state": "COMPLETE",
         "run_id": "7fce6736-8330-48b6-9f6d-e0f5fd9a67f6"
      },
      {
         "state": "COMPLETE",
         "run_id": "4e77d89e-0ce8-4b6e-bc0c-d53fb2ef2747"
      },
      {
         "state": "COMPLETE",
         "run_id": "352cc717-0ec7-468c-b60b-ab3aba2a80a0"
      },
      {
         "state": "EXECUTOR_ERROR",
         "run_id": "b05e0f28-dfb5-4ffb-aaf1-40fa79912229"
      },
      {
         "state": "EXECUTOR_ERROR",
         "run_id": "0607cbac-477e-40b6-8ff3-b3d3aa0999af"
      },
      {
         "state": "EXECUTOR_ERROR",
         "run_id": "fd96625a-77eb-4faf-bfd3-24a083d2b17e"
      },
      {
         "state": "EXECUTOR_ERROR",
         "run_id": "488c50da-62a1-461d-8f6d-abe16b58aa42"
      },
      {
         "state": "EXECUTOR_ERROR",
         "run_id": "86a0801c-decd-4ffa-9d0f-db08f9954db5"
      },
      {
  

{
   "runs": [
      {
         "state": "COMPLETE",
         "run_id": "9b7cd7ef-f08e-49d7-bdbf-51b5902c5863"
      },
      {
         "state": "COMPLETE",
         "run_id": "abc005b1-754f-44ab-9565-388d6f1e1abb"
      },
      {
         "state": "COMPLETE",
         "run_id": "87f9aa14-de70-44de-bfaf-3f65f7521b40"
      },
      {
         "state": "COMPLETE",
         "run_id": "235f63b7-c720-4d52-818f-e7b7196d423d"
      },
      {
         "state": "COMPLETE",
         "run_id": "ace1f0c1-1925-40e3-9e08-9fa728b0c8ae"
      },
      {
         "state": "COMPLETE",
         "run_id": "c61cb63c-ed74-4382-b5db-161e2099dad9"
      },
      {
         "state": "COMPLETE",
         "run_id": "931c3c73-4848-4889-9f99-1d862440d9fc"
      },
      {
         "state": "COMPLETE",
         "run_id": "cb4713f5-568e-4019-b86b-a3a27140dc8e"
      },
      {
         "state": "COMPLETE",
         "run_id": "48d44c09-6d90-4318-81e1-16b5fe5b1d68"
      },
      {
         "state": "COMPLETE",
  

{
   "runs": [
      {
         "state": "COMPLETE",
         "run_id": "5a933a8b-0921-4216-beb8-d85a38a32edd"
      },
      {
         "state": "COMPLETE",
         "run_id": "97ae2f02-827d-45dc-971e-161a527aff0a"
      },
      {
         "state": "COMPLETE",
         "run_id": "16093ce2-e157-4148-9a2f-e2d051c9482a"
      },
      {
         "state": "COMPLETE",
         "run_id": "261a7530-a2ed-4cb3-b3a2-d21699f436e5"
      },
      {
         "state": "COMPLETE",
         "run_id": "bdd3fb57-769f-4112-b8cf-97b8c4b9bd4a"
      },
      {
         "state": "COMPLETE",
         "run_id": "9b281475-cb0c-496d-92f5-13b9f3cab965"
      },
      {
         "state": "COMPLETE",
         "run_id": "f5675d2a-4735-4076-b217-accdb6d20858"
      },
      {
         "state": "COMPLETE",
         "run_id": "db841fd9-586e-4b9c-a3b4-e7d073dbb583"
      },
      {
         "state": "COMPLETE",
         "run_id": "bf37f355-4fc0-4684-810a-2e8954518e1a"
      },
      {
         "state": "EXECUTOR_ERRO

{
   "runs": [
      {
         "state": "COMPLETE",
         "run_id": "e7564d8a-5d2a-4069-9716-9d894177f7a2"
      },
      {
         "state": "CANCELED",
         "run_id": "902026a8-6770-4932-9b36-b33a302e7618"
      },
      {
         "state": "CANCELED",
         "run_id": "67dfd7df-6a10-4ad8-b574-f1e77ad8ee4f"
      },
      {
         "state": "CANCELED",
         "run_id": "724eb8c4-56c1-4298-81a6-7f4288bcdbd6"
      },
      {
         "state": "CANCELED",
         "run_id": "4b3852c0-88b8-4a72-a428-780b59753c0d"
      },
      {
         "state": "CANCELED",
         "run_id": "c4497ce8-76c9-4443-b50e-b103bd523179"
      },
      {
         "state": "CANCELED",
         "run_id": "5bccac8d-46c3-45ac-bc4c-b947e0bf4eff"
      },
      {
         "state": "CANCELED",
         "run_id": "d970b2f1-de37-4df1-b619-1987f1ca7574"
      },
      {
         "state": "CANCELED",
         "run_id": "24b66130-d7bd-4b34-97a5-f68ad8b7452c"
      },
      {
         "state": "COMPLETE",
  

{
   "runs": [
      {
         "state": "COMPLETE",
         "run_id": "11ebd3e5-de7d-453e-bf2a-42d7718930d4"
      },
      {
         "state": "COMPLETE",
         "run_id": "baef8c37-d953-48d2-8923-5b3214e49d2a"
      },
      {
         "state": "COMPLETE",
         "run_id": "2ea400ca-ffcb-4810-a586-aaf95cd8601e"
      },
      {
         "state": "EXECUTOR_ERROR",
         "run_id": "5e58cb19-a7f5-4abe-aa77-4cef72176da8"
      },
      {
         "state": "EXECUTOR_ERROR",
         "run_id": "0864087b-7995-4005-858b-45be32c6e1b7"
      },
      {
         "state": "EXECUTOR_ERROR",
         "run_id": "cfb6a69f-b512-4ca6-8553-5c759d367515"
      },
      {
         "state": "COMPLETE",
         "run_id": "413ee42d-b3f5-47de-9ef8-9764f8a00c10"
      },
      {
         "state": "COMPLETE",
         "run_id": "28f56ded-0f64-44cc-ae4c-e3228b19e58f"
      },
      {
         "state": "COMPLETE",
         "run_id": "ef691c4e-0c4b-49af-a833-ee8b09107ada"
      },
      {
         "stat

,run_id,start,state,type


The above instantiates a client for the SevenBridges Cancer Genomics Cloud (CGC ). 

### Checking a previous run
For information we'll first use the client to get the details of a task that was run from the CGC user interface
The getTaskStatus function below is simply a wrapper around https://cgc-ga4gh-api.sbgenomics.com/ga4gh/wes/v1/runs/{run_id} which deals with authentication, passing and retrieving the request. That  gives some clues about how to fill out a request to submit the same task via WES instead of the UI.

It's worth noting that though DRS was not used at all to create the task within the UI the file paths in the WES response do use a DRS notation for them.

In [3]:
cl.getTaskStatus('efb35efc-ce14-4b2b-84fe-18df78948c04', verbose=True)

Get request sent to: https://cgc-ga4gh-api.sbgenomics.com/ga4gh/wes/v1/runs/efb35efc-ce14-4b2b-84fe-18df78948c04
{
  "request": {
    "tags": {},
    "workflow_params": {
      "name": "CNest test run - 05-04-22 20:28:49",
      "project": "forei/cnest",
      "inputs": {
        "bed": {
          "path": "drs://cgc-ga4gh-api.sbgenomics.com/626bfb1bf26c93517368984e",
          "name": "hg38.1kb.baits.bed",
          "class": "File"
        },
        "project": "test_proj"
      }
    },
    "workflow_type": "CWL",
    "workflow_engine_params": {}
  },
  "state": "COMPLETE",
  "outputs": {
    "index_txt": {
      "path": "drs://cgc-ga4gh-api.sbgenomics.com/6272e29bf26c935173785796",
      "name": "index.txt",
      "class": "File"
    },
    "index_bed": {
      "path": "drs://cgc-ga4gh-api.sbgenomics.com/6272e29bf26c935173785798",
      "name": "index.bed",
      "class": "File"
    },
    "index_tab": {
      "path": "drs://cgc-ga4gh-api.sbgenomics.com/6272e29bf26c935173785794",
  

'COMPLETE'

Looking at that response gives some clues about how to edit the example provided in the documentation.

How the task looks in the UI is also helpful.
![alt text](SAMToolsTask.png "samtools task as shown in SevenBridges CGC UI")

## Running the same compute via the WES API
Filling out the body for a WES request to run the same thing, the project information is easy to work out. Inputs too seem pretty straightforward. Even though it's not present in the status above it's also pretty obvious that workflow_url should be the URI for the samtools stats app in my gecco project. The only tricky one was workflow_type_version. The log for the task run via the UI gives us a clue for that; job.json contains "cwlVersion" : "sbg:draft-2".

With all that we come up with the following for the request.

In [7]:
params = {
    "project": "forei/cnest",
        "inputs": {
        "bed": {
          "path": "drs://cgc-ga4gh-api.sbgenomics.com/626bfb1bf26c93517368984e",
          "name": "hg38.1kb.baits.bed",
          "class": "File"
        },
        "project": "test_proj"
      }
 
    }


## Calling WES from Python

Now we have formulated the body in the way that it can be passed to a client function as follows.

In [10]:
import json
run_id= cl.runGenericWorkflow(
    workflow_url='sbg://forei/cnest/cnest-test/5',
    workflow_params = json.dumps(params),
    workflow_type = "CWL",
    workflow_type_version = "sbg:draft-2",
    verbose=False
)
run_id

'0019e780-4681-430a-8d56-3b8f07ed8764'

In [11]:
cl.getTaskStatus(run_id)

'QUEUED'

## Getting the results - via DRS
Once the run is complete, further steps can use DRS to obtain the file output from the workflow.

In [10]:
runLog = cl.GetRunLog(run_id)
runLog['outputs']

{'statistics': {'path': 'drs://cgc-ga4gh-api.sbgenomics.com/600377ebe4b06d0cc270fc21',
  'name': '_12_117438.recal.cram.stats.txt',
  'class': 'File'}}

In [11]:
resultsDRSID = runLog['outputs']['statistics']['path']
resultsDRSID

'drs://cgc-ga4gh-api.sbgenomics.com/600377ebe4b06d0cc270fc21'

We'll pass over the question of how one would determine which DRS server that URI needs to be sent to because
* In this case it's fairly obvious - it's the CGC DRS Server
* We want to get something up and working
* There are other things we should consider when dealing with metaresolvers

Add to to-do list: a notebook on Metaresolvers

In [12]:
from fasp.loc import sbcgcDRSClient
drsClient = sbcgcDRSClient('~/.keys/sevenbridges_keys.json', 's3')

### DRS GetObject
Here's how we then get details of the file. Note that here only the id portion of the DRS URI is being passed. It is the job of a metaresolver to look at that URI and to determine where to send the id. As noted, we are passing up on the opportunity to use a metaresolver and putting in the id manually.

In [13]:
fileDetails = drsClient.getObject('5ffe65dee4b0eeecd99a2b39')
fileDetails

{'id': '5ffe65dee4b0eeecd99a2b39',
 'name': '_3_117438.recal.cram.stats.txt',
 'size': 111394,
 'checksums': [{'type': 'etag',
   'checksum': '347d17ba60392492bff1689cae4355b5-1'}],
 'self_uri': 'drs://cgc-ga4gh-api.sbgenomics.com/5ffe65dee4b0eeecd99a2b39',
 'created_time': '2021-01-13T03:15:42Z',
 'updated_time': '2021-01-13T03:15:42Z',
 'mime_type': 'application/json',
 'access_methods': [{'type': 's3',
   'region': 'us-east-1',
   'access_id': 'aws-us-east-1'}]}

In [14]:
url = drsClient.getAccessURL('5ffe65dee4b0eeecd99a2b39','s3')

### Downloading the file
Now we can use the url obtained to download the file. We'll create a small function to encapsulate the download.

In [15]:
import requests
import os
def download(url, file_path):
    with open(os.path.expanduser(file_path), "wb") as file:
        response = requests.get(url)
        file.write(response.content)

In [16]:
fullPath = '~/Downloads/' + fileDetails['name']
download(url, fullPath)

## Why we need a Metaresolver
Just to prove that to use DRS URIs we need a metaresolver. Here's what happens when we try asking the DRS Server to resolve the full DRS URI.

In [ ]:
drsClient.getObject('drs://cgc-ga4gh-api.sbgenomics.com/5ffe65dee4b0eeecd99a2b39')

That we get an error (404) might seem weird or obtuse behavior for at least couple of reasons:
* The DRS server clearly knows that is the URI for that file. It tells us so in the self_uri attribute
* The WES server from the same organization was quite happy with the full URI

However this behavior is correct according to the spec (double check that). A DRS Server resolves only the identifiers that are local to it.

A metaresolver would be needed for resolving compact URIs too.

In [ ]:
Get the details of the run of step 2

In [15]:
cl.getTaskStatus('84904f51-04a0-426d-850d-9fb0f1b0b331', verbose=True)

Get request sent to: https://cgc-ga4gh-api.sbgenomics.com/ga4gh/wes/v1/runs/84904f51-04a0-426d-850d-9fb0f1b0b331
{
  "request": {
    "tags": {},
    "workflow_params": {
      "name": "CNest step2 run - 05-08-22 21:38:11",
      "project": "forei/cnest",
      "inputs": {
        "index_txt": {
          "path": "drs://cgc-ga4gh-api.sbgenomics.com/627653faf26c9351737f92ac",
          "name": "index.txt",
          "class": "File"
        },
        "index_bed": {
          "path": "drs://cgc-ga4gh-api.sbgenomics.com/627653faf26c9351737f92ae",
          "name": "index.bed",
          "class": "File"
        },
        "project": "test_proj",
        "index_tab": {
          "path": "drs://cgc-ga4gh-api.sbgenomics.com/627653faf26c9351737f92af",
          "name": "index_tab.txt",
          "class": "File"
        },
        "sample": "test_bam2",
        "bam": {
          "path": "drs://cgc-ga4gh-api.sbgenomics.com/6272e873d125a52cff9b0247",
          "name": "TCGA-3X-AAVA-01A-11R-A41D-

'COMPLETE'

In [22]:
params = {
    "project": "forei/cnest",
    "inputs": {
        "index_txt": {
          "path": "drs://cgc-ga4gh-api.sbgenomics.com/627653faf26c9351737f92ac",
          "name": "index.txt",
          "class": "File"
        },
        "index_bed": {
          "path": "drs://cgc-ga4gh-api.sbgenomics.com/627653faf26c9351737f92ae",
          "name": "index.bed",
          "class": "File"
        },
        "project": "test_proj",
        "index_tab": {
          "path": "drs://cgc-ga4gh-api.sbgenomics.com/627653faf26c9351737f92af",
          "name": "index_tab.txt",
          "class": "File"
        },
        "sample": "test_bam",
        "bam": {
          "path": "drs://cgc-ga4gh-api.sbgenomics.com/6272e873d125a52cff9b0247",
          "name": "TCGA-3X-AAVA-01A-11R-A41D-13_mirna_gdc_realn.bam",
          "secondaryFiles": [
            {
              "path": "drs://cgc-ga4gh-api.sbgenomics.com/6272ec5df26c93517378730b",
              "name": "TCGA-3X-AAVA-01A-11R-A41D-13_mirna_gdc_realn.bam.bai",
              "class": "File"
            }
          ],
          "class": "File"
        }
      }
    }



In [23]:
#import json
run_id= cl.runGenericWorkflow(
    workflow_url='sbg://forei/cnest/cnest-step2/14',
    workflow_params = json.dumps(params),
    workflow_type = "CWL",
    workflow_type_version = "sbg:draft-2",
    verbose=False
)
run_id

'c1f9555a-cfb3-4053-bf7c-b7549846c34f'

Can we run the above with a bam file from a URL obtained via DRS?

We'll try with the Gen3 id of the same file as above



In [20]:
params['bam'] = {
          "path": "drs://cgc-ga4gh-api.sbgenomics.com/6272e873d125a52cff9b0247",
          "name": "TCGA-3X-AAVA-01A-11R-A41D-13_mirna_gdc_realn.bam",
          "secondaryFiles": [
            {
              "path": "drs://cgc-ga4gh-api.sbgenomics.com/6272ec5df26c93517378730b",
              "name": "TCGA-3X-AAVA-01A-11R-A41D-13_mirna_gdc_realn.bam.bai",
              "class": "File"
            }
          ],
          "class": "File"
        }


In [21]:
params

{'project': 'forei/cnest',
 'inputs': {'index_txt': {'path': 'drs://cgc-ga4gh-api.sbgenomics.com/627653faf26c9351737f92ac',
   'name': 'index.txt',
   'class': 'File'},
  'index_bed': {'path': 'drs://cgc-ga4gh-api.sbgenomics.com/627653faf26c9351737f92ae',
   'name': 'index.bed',
   'class': 'File'},
  'project': 'test_proj',
  'index_tab': {'path': 'drs://cgc-ga4gh-api.sbgenomics.com/627653faf26c9351737f92af',
   'name': 'index_tab.txt',
   'class': 'File'},
  'sample': 'test_bam2',
  'bam': {'path': 'drs://cgc-ga4gh-api.sbgenomics.com/6272e873d125a52cff9b0247',
   'name': 'TCGA-3X-AAVA-01A-11R-A41D-13_mirna_gdc_realn.bam',
   'secondaryFiles': [{'path': 'drs://cgc-ga4gh-api.sbgenomics.com/6272ec5df26c93517378730b',
     'name': 'TCGA-3X-AAVA-01A-11R-A41D-13_mirna_gdc_realn.bam.bai',
     'class': 'File'}],
   'class': 'File'}},
 'bam': {'path': 'drs://cgc-ga4gh-api.sbgenomics.com/6272e873d125a52cff9b0247',
  'name': 'TCGA-3X-AAVA-01A-11R-A41D-13_mirna_gdc_realn.bam',
  'secondaryFiles